In [18]:
import numpy as np
import evaluate
import random
import pandas as pd
import datasets
import subprocess
import time
from IPython.display import display,HTML
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer

### 任务一：文本分类任务

In [5]:
# 读取数据
dataset = load_dataset("yelp_review_full")

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [9]:
# 查看样例
def show_random_elements(dataset,num_examples=10):
    assert num_examples <= len(dataset),"不能输出超过数据集数量的元素"
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0,len(dataset)-1)
        while pick in picks:
            pick = random.randint(0,len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column,type in dataset.features.items():
        if isinstance(type,datasets.ClassLabel):
            df[column] = df[column].transform(lambda i:type.names[i])

    display(HTML(df.to_html()))

show_random_elements(dataset["train"])

,label,text
0,2 star,Decor was nice but the food needs flavor. If you like bland Mexican this is for you.
1,5 stars,Came in to take my Bride for our simple night out on our 20th Wedding Anniversary!\nA Pleasant young man (ISIA) greeted us in the front and seated us in a very nice roomy booth. Then came our server (SERGIO) who is also Nice and Pleasant and helped us with the menu and made plenty of suggestions to make our picks really easy. We ordered the Wonton soup with egg noodles which is alright and the Highlight of our dinner which is the Hakka Rice! Oh man! That was some Beautiful Food! It's a must try dish! Everyone seems to be on top of their game! (Service that is!) A fine gentleman approach us (TONY) and checked and see how we are doing and Congratulated us on our Anniversary and gave us something Special which we really enjoyed! To all of You guys that I mentioned THANK YOU ALL for a Wonderful evening at KU Noodle. We Will Be Back!
2,2 star,Slides are cool but the resort itself is lacking a lot. \nAsked for the directions for the gym and didn't give me clear directions.\nAC was super loud.\nKids running around past 11pm. \nEverything is overpriced.
3,5 stars,"$20 all you can eat sushi, how can you go wrong? The place is not too big and has a homey feel. The workers there are all awesome. I mean really awesome. One of my new favorite habits. Going to try to make it at least once a month."
4,3 stars,"The decor is reflective of The Malaysian tropics, island style. Wood accents throughout entire restaurant along with a foosball and pool table in the back. It's a warm casual ambience, laid back. All the employees are friendly and appear joyful which elevates the mood of the diners as well. The dishes lacked flavor and were average. The \""Roti Canai\"" and curry items on the menu I believe were the standout stars! I didn't get the chance to try their whole fish or seafood items so I guess I have to come back! The stir fried noodles I ordered lacked flavor, I had to add more soy sauce. Overall, the menu has abundance of seafood items in which I hope I get to try later on."
5,5 stars,"I'm not a \""shoe girl\"". I have also, recently, been condemned to wearing one pair of running trainers *everywhere* due to a really bad knee injury combined with terrible biomechanics (read: I have bow legs). Until I can get insoles, trainers were my one and only footwear.\n\nBut now the sun is out, my knee is better, and I am looking at my current shoe collection thinking \""oh mah gah, these are all SERIOUSLY old\"". My one pair of ballet flats for summer were \u00a312 New Look numbers that used to be metallic gold. Now they're just off-white. My two pairs of cheap trainers (no brand) have stains on them of I don't know what. I have seen my shoes in a whole new light. A shoe lady I may never become, but hell, I NEED new shoes.\n\nBrief autobiography of my feety matters done, now to the review at hand. I rarely review chains, and even less frequently (in fact, this could be a first) give them the full sexy five stars. Schuh is a store I rarely frequent too. I'm not a fan of 'brands' really, and prefer boutiquey numbers. I also have ridiculously narrow feet (not small, they're just long and thin) and most shoes are like boats on me and fall off.\n\nBut when I stubbed my toe AGAIN in my battered gold ballet flats and realised the sole was hanging off, it happened right outside Schuh, and I took it as a sign. On wandering in I realised \""wow this place is big and there are a LOT of shoes\"".\n\nMy only other Schuh experience was the one on North Bridge which is very small, and the two stores in Glasgow which are 'ok' but I always left disinterested.\n\nIt could be fashion is turning my way, or that in my new found 'I can wear non-trainers!' state of mind I saw shoes in a whole new way. But they had a lot of different brands in here, and almost all of the offerings I liked!\n\nThe staff were also really attentive. It was a busy Bank Holiday M

In [7]:
# 数据预处理
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"],padding="max_length",truncation=True)

tokenized_datasets = dataset.map(tokenize_function,batched=True)

In [10]:
# 查看处理后的数据
show_random_elements(tokenized_datasets["train"],num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,5 stars,"Ok, I'm not a vegan. In fact, when my hubby picked me up for lunch and surprised me by taking me to this little hidden restaurant, I was a wee bit nervous about finding something on the menu I would like. Instead, I just returned to my office, handed my co-worker the menu, insisted she go eat there for lunch, and sat down to write this review. Yup, it was THAT good. \n\nWhen we walked in, we were greeted by Areeya who informed us that the salads and pizzas (on gluten free crust) were something that were favorited by most since we were new to the place. So, that's exactly what we got, a half House Salad with the House dressing and a full Traditional Pizza. We got our salad first and it was a smorgasbord of fresh veggies (avocado, bell pepper, cucumber, mixed greens, sprouts) and nuts mixed with the house dressing which tasted fresh and garlicy. Then came the pizza, which the menu claims is 'unlike any you've ever had'... and this is why that statement is true: The gluten free crust isn't a thin piece tasteless cardboard like one would expect, it is a thick, delicious, buckwheat crust that was the perfect consistency for piling on the delicious pesto marinara, basil, cauliflower, almonds, & walnuts. The flavors were extraordinary! \n\nI'm not a total health nut and I'm definitely not a vegan but this place serves up DELICIOUS healthy food and I cannot wait to return to try more items from the menu.\n\nTIP: If you're ordering for lunch, call ahead and place your order - It did take a little while to get the food.","[101, 23330, 117, 146, 112, 182, 1136, 170, 1396, 3820, 119, 1130, 1864, 117, 1165, 1139, 10960, 2665, 3015, 1143, 1146, 1111, 5953, 1105, 3753, 1143, 1118, 1781, 1143, 1106, 1142, 1376, 4610, 4382, 117, 146, 1108, 170, 1195, 1162, 2113, 5604, 1164, 4006, 1380, 1113, 1103, 13171, 146, 1156, 1176, 119, 3743, 117, 146, 1198, 1608, 1106, 1139, 1701, 117, 3541, 1139, 1884, 118, 7589, 1103, 13171, 117, 6744, 1131, 1301, 3940, 1175, 1111, 5953, 117, 1105, 2068, 1205, 1106, 3593, 1142, 3189, 119, 10684, 1643, 117, 1122, 1108, 157, 11612, 1942, 1363, 119, 165, 183, 165, 183, 2924, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


In [7]:
# 模型微调
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",num_labels=5)

# 设置模型参数
model_dir = "models/bert-base-cased-finetune-yelp-full"
 
training_args = TrainingArguments(output_dir = model_dir,
                                  evaluation_strategy = "epoch",
                                 per_device_train_batch_size=16,
                                 num_train_epochs=3,
                                 logging_steps=1000)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 设置评估指标
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits,labels = eval_pred
    predictions = np.argmax(logits,axis=-1)
    return metric.compute(predictions=predictions,references=labels)

In [8]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

由于65万数据需要43个小时，太耗时了，现用1万样本进行训练。

In [9]:
# 数据抽样
train_dataset_1w = tokenized_datasets["train"].shuffle(seed=10).select(range(10000))
eval_dataset_1w = tokenized_datasets["test"].shuffle(seed=10).select(range(10000))

In [10]:
# 模型训练
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_10w,
    eval_dataset=eval_dataset_1w ,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


在后台运行模型训练文件，保存模型"models/bert-base-cased-finetune-yelp-1w"

#### 比较利用1千和1万样本集训练的模型在测试集上的效果

In [4]:
model_dir_base = "models/bert-base-cased-finetune-yelp-1k"
model_dir = "models/bert-base-cased-finetune-yelp-1w"

#加载本地保存的模型
trained_model_base = AutoModelForSequenceClassification.from_pretrained(model_dir_base)
trained_model = AutoModelForSequenceClassification.from_pretrained(model_dir)

In [11]:
# 抽样验证集
test_dataset = tokenized_datasets["test"].shuffle(seed=10).select(range(10000))

In [14]:
training_args = TrainingArguments(output_dir = model_dir,
                                  evaluation_strategy = "epoch",
                                 per_device_train_batch_size=16,
                                 num_train_epochs=3,
                                 logging_steps=1000)

# 创建 Trainer 对象
trainer_base = Trainer(
    model=trained_model_base,
    args=training_args,
    eval_dataset=test_dataset ,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [20]:
trainer_base.evaluate()

{'eval_loss': 1.0299875736236572,
 'eval_accuracy': 0.5487,
 'eval_runtime': 288.3609,
 'eval_samples_per_second': 34.679,
 'eval_steps_per_second': 4.335}

In [19]:
# 创建 Trainer 对象
trainer = Trainer(
    model=trained_model,
    args=training_args,
    eval_dataset=test_dataset ,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [21]:
trainer.evaluate()

{'eval_loss': 1.121748685836792,
 'eval_accuracy': 0.6108,
 'eval_runtime': 286.4555,
 'eval_samples_per_second': 34.909,
 'eval_steps_per_second': 4.364}

**对比用1千训练集微调的bert模型，用1万训练集微调的bert模型在1万测试集上的精确度从0.5487提升到0.6108。**